# 式

## 式とは何か？

数学では、整数または実数変数に対する二項演算または単項演算を考えます。例えば、$x + y$や$x^2$です。`jijmodeling`で定義された変数に対しても対応する演算が可能であり、その結果は「式」と呼ばれます。

In [1]:
import jijmodeling as jm

x = jm.BinaryVar("x")
y = jm.BinaryVar("y")
z = x + y
w = x ** 2

`x`と`y`は`BinaryVar`オブジェクトであり、`z`と`w`は表現です。これは「従属変数」とは少し異なります。`z`は、私たちが考えている問題の変数ではありません。

![Expression Tree](./assets/expressions_01.png) <!-- https://docs.google.com/drawings/d/1g9hdbLD-nRSqPtLVU1NP1V-w0ITN0AAy4M4vV-0a4Ug/edit?usp=sharing -->

## 組み込み演算

Pythonの組み込み演算子（例：`+`）は、決定変数（例：`BinaryVar`）および`Placeholder`の両方に対してオーバーロードされています：

In [2]:
x = jm.BinaryVar("x")
y = jm.BinaryVar("y")
z = x + y
repr(z)  # "BinaryVar(name='x', shape=[]) + BinaryVar(name='y', shape=[])"

"BinaryVar(name='x', shape=[]) + BinaryVar(name='y', shape=[])"

これは象徴的なプロセスです。つまり、`z`は上記のような表現ツリーです。Pythonの組み込み関数`repr`を使用して式木の内容を表示することができます。また、Jupyter環境にいる場合は、LaTeXを使用してより美しい表示を得ることができます。

:::{admonition} 式の順序
:class: tip 
演算子のオーバーロードは線形演算に限定されず、`is_linear`関数を使用して式が線形項のみを含むかどうかを確認できます：

In [4]:
x = jm.BinaryVar("x")
jm.is_linear(x)  # True

True

In [5]:
w = x ** 2
jm.is_linear(w)  # False

False

式の次数を確認するために、`is_quadratic`や`is_higher_order`もあります。 :::

### 比較演算

等号演算子`==`および他の比較演算子（例：`<=`）も、等式および不等式制約を表すためにオーバーロードされています。2つの式木が同じかどうかを確認したい場合は、`is_same`関数を使用できます。

In [6]:
x = jm.BinaryVar("x")
y = jm.BinaryVar("y")
repr(x == y) # "BinaryVar(name='x', shape=[]) == BinaryVar(name='y', shape=[])"
jm.is_same(x, y) # False

False

## インデックスと総和

Pythonの組み込み`list`や`numpy.ndarray`と同様に、`jijmodeling`は多次元の決定変数やパラメータの要素にアクセスするためのインデックスをサポートしています：

In [ ]:
x = jm.BinaryVar("x", shape=(3, 4))
x[0, 2]

BinaryVar(name='x', shape=[NumberLit(value=3), NumberLit(value=4)])[NumberLit(value=0), NumberLit(value=2)]

:::{note}
`x[0][2]`と`x[0, 2]`は同じものを指しています。
:::

`x[0, 2]`も表現です。これは、単項演算子`**2`が適用された`x`の式木の場合と似ています。`x[0]`は、`x`に適用された「0番目の要素を取る」単項演算子`[0]`の式木です。添字は決定変数を含まない式であることができます：

In [8]:
x = jm.BinaryVar("x", shape=(3, 4))
n = jm.Placeholder("n")
x[2*n, 3*n]

BinaryVar(name='x', shape=[NumberLit(value=3), NumberLit(value=4)])[NumberLit(value=2) * Placeholder(name='n', ndim=0), NumberLit(value=3) * Placeholder(name='n', ndim=0)]

インデックス付けのための別のタイプの変数`Element`があります。これは、次のような内部変数$n$を表すために使用されます：

$$
\sum_{n=0}^{N-1} x_n .
$$

この総和を表現するために、jijmodelingは3つのステップを取ります：

In [9]:
N = jm.Placeholder("N")
x = jm.BinaryVar("x", shape=(N,))

# Introduce an internal variable $n$ with its range $[0, N-1]$
n = jm.Element('n', belong_to=(0, N))

# Create operand expression $x_n$ using indexing expression
xn = x[n]

# Sum up integrand expression $x_n$ about the index $n$
jm.sum(n, xn)

sum(Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0))), BinaryVar(name='x', shape=[Placeholder(name='N', ndim=0)])[Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0)))])

$$
\mathrm{jm.sum(\overbrace{n}^{subscript}, \underbrace{x[n]}_{operand})}
$$

:::{tip}
このような単純な総和は、次のように省略形で書くことができます:

In [10]:
N = jm.Placeholder('N')
x = jm.BinaryVar(name='x', shape=(N,))
sum_x = x[:].sum()

:::

`Element`オブジェクト自体が式として扱えるため、$\sum_n n x_n$を次のように書くことができます：

In [ ]:
jm.sum(n, n * x[n])

sum(Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0))), Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0))) * BinaryVar(name='x', shape=[Placeholder(name='N', ndim=0)])[Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0)))])

`jm.sum`の結果も表現です。これは、次のような同じ部分を繰り返す複雑な関数をモデル化するのに役立ちます：

$$
\left(\sum_{n=0}^{N-1} x_n \right)
\left(1 - \sum_{n=0}^{N-1} x_n \right)
$$

これをシンプルなコードに変換します：

In [ ]:
n = jm.Element('n', belong_to=(0, N))
sum_x = jm.sum(n, x[n])
sum_x * (1 - sum_x)

sum(Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0))), BinaryVar(name='x', shape=[Placeholder(name='N', ndim=0)])[Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0)))]) * (- sum(Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0))), BinaryVar(name='x', shape=[Placeholder(name='N', ndim=0)])[Element(name='n', belong_to=(NumberLit(value=0), Placeholder(name='N', ndim=0)))]) + NumberLit(value=1))

## 集合に沿ったイテレーション
数学モデルでは、しばしば集合$V$に対する総和を取ります。例えば、

$$ 
\sum_{v \in V} x_v .
$$

非連続のインデックス（例：$[1, 4, 5, 9]$や$[2, 6]$）が集合$V$として使用されます。

:::{tip}
このようなカスタム総和は、与えられたインデックス集合$V$に対するワンホット制約のような制約でよく見られます：

$$ 
\sum_{v \in V} x_v = 1
$$

このページで説明されているすべての表現は、Constraintオブジェクトでも使用できます
:::

In [13]:
N = jm.Placeholder('N')
x = jm.BinaryVar('x', shape=(N,))

# `V` is defined as a `Placeholder` object
V = jm.Placeholder('V', ndim=1)

# `V` can be passed as the argument of `belong_to` option
v = jm.Element('v', belong_to=V)

# This `Element` object can be used for summation
sum_v = jm.sum(v, x[v])

その後、実際のデータは次のように渡されます：

In [14]:
import jijmodeling_transpiler as jmt

problem = jm.Problem('Iterating over a Set')
problem += sum_v

instance_data = { "N": 10, "V": [1, 4, 5, 9]}
compiled_model = jmt.core.compile_model(problem, instance_data)

### Jagged配列

時々、$C_\alpha$のようなインデックスセットのシリーズを考慮する必要があります。例えば、Kホット制約を課すために：

$$
\sum_{i \in C_\alpha} x_i = K_\alpha.
$$

これらのインデックスセットは異なる長さを持つことがあります。例えば、

$$
\begin{align*}
C_1 &= [1, 4, 5, 9], \\
C_2 &= [2, 6], \\
C_3 &= [3, 7, 8]
\end{align*}
$$

`Placeholder`オブジェクトを使用して、このようなジャギー配列を表現できます：

In [15]:
N = jm.Placeholder('N')
x = jm.BinaryVar('x', shape=(N,))

# Define as 2-dim parameter
C = jm.Placeholder('C', ndim=2)

# Number of K-hot constraints
# Be sure we cannot take `C.len_at(1)` since `C` will be a jagged array
M = C.len_at(0, latex="M")

K = jm.Placeholder('K', ndim=1)

# Usual index
a = jm.Element(name='a', belong_to=(0, M), latex=r"\alpha")
# `belong_to` can take expression, `C[a]`
i = jm.Element(name='i', belong_to=C[a]) 

# K-hot constraint
k_hot = jm.Constraint('k-hot_constraint', jm.sum(i, x[i]) == K[a], forall=a)

その後、実際のデータは次のように渡されます：

In [16]:
problem = jm.Problem('K-hot')
problem += k_hot

instance_data = {
    "N": 4,
    "C": [[1, 4, 5, 9],
          [2, 6],
          [3, 7, 8]],
    "K": [1, 1, 2],
}
compiled_model = jmt.core.compile_model(problem, instance_data)

## 複数のインデックスに対する総和

複数の総和を計算するために、次のように式で表すことができます：

$$
\sum_{i, j} Q_{ij} x_{ij}
$$

これは`jijmodeling`で次のように実装できます。

In [17]:
# set variables
Q = jm.Placeholder('Q', ndim=2)
I = Q.shape[0]
J = Q.shape[1]
x = jm.BinaryVar('x', shape=(I, J))
i = jm.Element(name='i', belong_to=(0, I))
j = jm.Element(name='j', belong_to=(0, J))
# compute the sum over the two indices i, j
sum_ij = jm.sum([i, j], Q[i, j]*x[i, j])

複数の総和がある場合、複数の`jm.sum`を省略して、添字をリスト`[subscript1, subscript2, ...]`にすることができます。 もちろん、これは$\sum_{i, j} = \sum_{i} \sum_{j}$と書くことができることがわかります。

In [18]:
sum_ij = jm.sum(i, jm.sum(j, Q[i, j]*x[i, j]))

## 条件付き総和

可能なインデックスの全範囲にわたる総和に加えて、次のようにインデックスが特定の条件を満たす場合にのみ総和を取ることができます。

$$
\sum_{i<U} x_i 
$$

これは`jijmodeling`を使用して次のように実装できます。

In [19]:
# Define variables to be used
I = jm.Placeholder('I')
x = jm.BinaryVar('x', shape=(I,))
i = jm.Element(name='i', belong_to=(0, I))
U = jm.Placeholder('U')
# Calculate sum only for parts satisfying i<U
sum_i = jm.sum((i, i<U), x[i])

`jm.sum`のインデックスを指定する部分では、次のようにタプルを使用して`(index, condition)`を指定します。

$$
\mathrm{jm.sum((\underbrace{i}_{index}, \overbrace{i<U}^{condition}), \underbrace{x[i]}_{operand})}
$$

比較演算子`<`、`<=`、`>=`、`>`、`==`、`!=`および論理演算子`&`、`|`およびそれらの組み合わせを、添字が満たす条件式に使用できます。例えば、

$$
\sum_{\substack{i < U \\ i!=N}} d_i x_i
$$

は次のように実装できます。

In [20]:
# Define variables to be used
d = jm.Placeholder('d', ndim=1)
I = d.shape[0]
x = jm.BinaryVar('x', shape=(I,))
U = jm.Placeholder('U')
N = jm.Placeholder('N')
i = jm.Element(name='i', belong_to=(0, I))
# Calculate sum only for the part satisfying i<U and i≠N
sum_i = jm.sum((i, (i<U)&(i!=N)), d[i]*x[i])

$$
\mathrm{jm.sum((\underbrace{i}_{subscript}, \overbrace{(i<U)}^{condition 1} \underbrace{\&}_{logical operator} \overbrace{(i!=N)}^{condition 2}), \underbrace{d[i]*x[i ]}_{operand})}
$$

### 複数条件付きの総和
複数の添字に対する総和演算に条件がある場合、例えば

$$
\sum_{\substack{i>L \\ i!=N}} \sum_{j<i} R_{ij} x_{ij}
$$

次のような式を実装することができます。

In [21]:
# Define variables to be used
R = jm.Placeholder('R', ndim=2)
I = R.shape[0]
J = R.shape[1]
x = jm.BinaryVar('x', shape=(I, J))
i = jm.Element(name='i', belong_to=(0, I))
j = jm.Element(name='j', belong_to=(0, J))
N = jm.Placeholder('N')
L = jm.Placeholder('L')
# Calculate sum only for the part satisfying i>L and i!=N and j<i
sum_ij = jm.sum([(i, (i>L)&(i!=N)), (j, j<i)], R[i, j]*x[i, j])

`jm.sum`でインデックスを指定する部分は、`[[(index 1, condition of index 1), (index 2, condition of index 2), ...]]`のように置き換える必要があります。`[(index1, condition of index1), (index2, condition of index2), ...]`のように見えるようにすることで、各インデックスに条件を付けた複数の総和演算を記述できます。

:::{caution} `[(j, j<i), (i, (i>L)&(i!=N))]`はエラーが発生します。なぜなら、$i$はまだ$j<i$の部分で定義されていないからです。 これは$\sum_{\substack{i>L \\ i!=N}} \left( \sum_{j<i} \cdots \right)$のように式で書くことができますが、$\sum_{j<i} \left( \sum_{\substack{i>L \\ i!=N}} \cdots \right)$は書くことができないことに対応します。複数の総和において添字に条件を課す順序に注意してください。 :::